
[<--energy](./energy.ipynb)
    
     scp -r root@apps.sitebuilt.net:/home/jupyter/public_html/ /home/jupyter/
     the one source of truth is on apps.sitebuilt


### alt calcs - annual fuel use, degree days and heating load
To check on the Manual J heatLoss calculations, below you will find links to equivalent calculations in which design load is calculated from 
from historic averages of gallons of oil burned, degree days for Boston and the BTU per fuel unit. 

##### refs
* [intel cloud jupyter](https://console.cloud.intel.com)

* Heat Loss Calculations and Principles - pdf in /onedrive/energy
* degree-days Boston [current](https://massenergymarketers.org/resources/degree-days/boston/) - [23yr table](https://www.weather.gov/wrh/Climate?wfo=box) -  [query and API](https://www.degreedays.net)
* [projecting heating costs from btu/hr and degree days](https://forum.heatinghelp.com/discussion/131998/calculating-heating-costs-math-problem)
* [calculating home energy tutorial](http://hyperphysics.phy-astr.gsu.edu/hbase/thermo/heatloss.html)

#### For 255 chestnut Ave apt3 1-4
##### verifying manual J heatLoss
In general

(1)$$\frac{units}{yr} = \frac{heatLoss \frac{Btu}{hr} * 24\frac{hr}{day} * Degree \frac{T-day}{yr}}{\frac{Btu}{unit} * \Delta T * eff. }$$

solving for heatLoss@5F 

(1a)$$heatLoss \frac{Btu}{hr} = \frac{ \frac{units}{yr} * \frac{Btu}{unit} * \Delta T * eff.  }{24 \frac{hr}{day} * degree \frac{T-day}{yr}}$$

The heatLoss @ 5F was derived from manual J calculations.

To establish the relationship between the `historic fuel use` and `heatLoss @ 5`&deg;F we tweaked boiler efficiency until we got gallons to agree to historic averages.

(2)$$onePipeSteamBoilerOil\frac{units}{yr} = \frac{HeatLoss \frac{Btu}{hr} * 24\frac{hr}{day} * Degree \frac{T-day}{yr}}{Design \Delta T * Efficiency * \frac{Btu}{unit}}$$

In [40]:
costPerKwh =.27
BtuPerKw = 3412
BtuPerTon =12000
ElKwhPerYr = 1433 # actual fuel use for apt 3 2022-2023
costPerGal = 4.50
heatLoss = 13000 # apt manual J
deltaT = 65
degreeDays =4883 # used in calcs below
dd2021 = 5071
dd2122 = 5004
dd2223 = 4883 # degree days 2022-23
dd1823 = 5263 # 5 yr avg. degree day 2018 - 2023
dd9212 = 5714 # 20 year average 
effOil = .80 #of onepipe steam oil burner
BtuPerGal = 139000
galPerYr =(heatLoss * 24 * degreeDays)/(deltaT * effOil *BtuPerGal)

print(f"{galPerYr =:.0f} gal/yr @ {effOil = :.0%}")

galPerYr =211 gal/yr @ effOil = 80%


This seems to be an reasonable efficiency value for The 35 year old Burnham a one-pipe steam oil fired system . So the calculated manual J heatLoss appears to be accurate. 

##### Predicting heat pump efficiency given electrical use for apt given heatLoss and degreeDays
For Apt 3 winter 22-23...

(3)$$heatPump\frac{kwh}{yr} = \frac{HeatLoss \frac{Btu}{hr} * 24\frac{hr}{day} * Degree \frac{T-day}{yr}}{ \Delta T * eff. * \frac{Btu}{kw}}$$

Solving for efficiency

(3a)$$eff = \frac{HeatLoss \frac{Btu}{hr} * 24\frac{hr}{day} * Degree \frac{T-day}{yr}}{heatPump\frac{kwh}{yr} * \Delta T * \frac{Btu}{kw}}$$

In [2]:
effHP = (heatLoss * 24 * degreeDays)/(ElKwhPerYr * deltaT * BtuPerKw)

print(f"Apt 3 HeatPump electricity use 22-23 = {ElKwhPerYr:,.0f} kw/yr \
  \nimplies a heat pump efficiiency of {effHP:.1f} using \
  heatLoss @ 5F of {heatLoss:,.0f} Btu/hr")

Apt 3 HeatPump electricity use 22-23 = 1,433 kw/yr   
implies a heat pump efficiiency of 4.8 using   heatLoss @ 5F of 13,000 Btu/hr


Knowing how many `kwh` apt3 used in a year, we can predict total `Btu/yr`. (Not sure what the efficiency number (effHP) represents relative to COP or HSPF)




#### Heat Pump Efficiency


[Heating seasonal performance factor HSPF](https://en.wikipedia.org/wiki/Heating_seasonal_performance_factor#:~:text=The%20higher%20the%20HSPF%20rating,a%20US%20Energy%20Tax%20Credit.)

(4)$$HSPF = \frac{Btu}{watt-hr} = \frac{Btu}{(1000*kwh)}$$

Solving fro kwh

(5)$$ kwh = \frac{Btu}{(1000*HSPF)}$$

Solving for Btu

(6) $$\frac{Btu}{yr} = 1000 * \frac{Kwh}{yr} * HSPF$$ 

Here we used HSP and fuel use to estimate total energy use in a year  Btu/yr. So for apt 3 in 22-23



In [3]:
HSPF = 10.4 # for LV181HV4
HSPFresist =3.4 # for electric resistance heat (100% efficient)
EffComparedToResist = HSPF/HSPFresist
EffComparedToOil = HSPF/(effOil * HSPFresist)
apt3BtuPerYr = 1000 * ElKwhPerYr * HSPF
print(f"using {HSPF = }, apt3BtuPerYr = {apt3BtuPerYr:,.0f} Btu when ElKwhPerYr = {ElKwhPerYr:,.0f}kwh \
  \nEffComparedToResist:  is {EffComparedToResist:.1f}\
 times more efficient than elec.resistance \
  \nEffComparedToOil:  is {EffComparedToOil:.1f}\
 times more efficient than oil")

using HSPF = 10.4, apt3BtuPerYr = 14,903,200 Btu when ElKwhPerYr = 1,433kwh   
EffComparedToResist:  is 3.1 times more efficient than elec.resistance   
EffComparedToOil:  is 3.8 times more efficient than oil


Another way to get Btu/yr is to estimate it from heatLoss @5F and degree days.

(7)$$\frac{Btu}{yr} = heatLoss \frac{Btu}{hr}*24 \frac{hr}{day} * \frac{1}{ \Delta T}*Degree \frac{T-days}{yr}$$


In [4]:
#heatLoss = 10000 //try a different heatLoss @5F

apt3BtuYr =heatLoss * 24 / deltaT * degreeDays
ratio = apt3BtuYr/apt3BtuPerYr 
print(f"By (7) using heatLoss@5F={heatLoss:,.0f} and {degreeDays=}: apt3BtuYr = {apt3BtuYr:,.0f} btu/yr \
  \nBy (6) using {ElKwhPerYr = :,.0f} and { HSPF=} {apt3BtuPerYr = :,.0f} Btu/yr\
  \nSo the apt3BtuYr(7)/apt3BtuPerYr(6) {ratio = :.2f}")


By (7) using heatLoss@5F=13,000 and degreeDays=4883: apt3BtuYr = 23,438,400 btu/yr   
By (6) using ElKwhPerYr = 1,433 and  HSPF=10.4 apt3BtuPerYr = 14,903,200 Btu/yr  
So the apt3BtuYr(7)/apt3BtuPerYr(6) ratio = 1.57


Coefficient of performance

(8)$$COP = \frac{Btu}{kw*3412\frac{Btu}{kw}}$$ 

In [5]:
COPa = apt3BtuYr / (ElKwhPerYr * 3412)
COPb = apt3BtuPerYr / (ElKwhPerYr * 3412)

print(f"With {heatLoss = :,.0f} Btu/hr & {ElKwhPerYr = :,.0f} kwh \
  \n{COPa =  :.1f} with {apt3BtuYr = :,.0f} \
  \n{COPb =  :.1f} with {apt3BtuPerYr = :,.0f} ")

With heatLoss = 13,000 Btu/hr & ElKwhPerYr = 1,433 kwh   
COPa =  4.8 with apt3BtuYr = 23,438,400   
COPb =  3.0 with apt3BtuPerYr = 14,903,200 


##### apt3 predicted costs comapared to actual costs for 2023

(5)$$ kwh = \frac{Btu}{(1000*HSPF)}$$

In [6]:
apt3predKwh = apt3BtuYr/ (1000 * HSPF)
apt3predCost = apt3predKwh * costPerKwh
apt3actualCost = ElKwhPerYr * costPerKwh

print(f"Uing {HSPF =} \nPREDICTED: {apt3BtuYr = :,.0f} Btu/yr\n\
  {apt3predKwh =:,.0f} kwh and apt3predCost = ${apt3predCost:,.2f} \
    \nACTUAL {apt3BtuPerYr = :,.0f} Btu/yr \
    \n  {ElKwhPerYr = :,.0f} kwh  and apt3actualCost = ${apt3actualCost:,.2f}")

Uing HSPF =10.4 
PREDICTED: apt3BtuYr = 23,438,400 Btu/yr
  apt3predKwh =2,254 kwh and apt3predCost = $608.50     
ACTUAL apt3BtuPerYr = 14,903,200 Btu/yr     
  ElKwhPerYr = 1,433 kwh  and apt3actualCost = $386.91


The PREDICTED energy used in a year (Btu/yr) used manual J heatLoss@5F, degree days. The manual J calcs were verified by heatLoss@5F calculated from fuel use

There is an inconsistency between PREDICTED annual energy use and cost compared to ACTUAL energy use and cost. This is not resolved. It is a good thing that the predictions were higher than actual, and it was also a good thing that the predicted energy use and cost was lower than the energy use and cost using oil. So the moral is using the prediction model is OK and you can maybe beat the predictions with good design and equipment selection.

#### 12 parley vale

1. Predict building heatLoss @5F from historic natural gas prices and degree days
2. Estimate building total Btu/yr from HeatLoss @ 5F and degree days
3. Estimate how many kwh a new heat pump system will need to provide total Btu/hr using HSPF
4. Compare new system \$/yr to old system \$/yr

#####  heat loss in stairways

[Understanding Thermal Comfort Impact and Air Movement
Around Open Stairs Through the Use of CFD Modeling](https://scholarworks.uark.edu/cgi/viewcontent.cgi?article=6015&context=etd)

##### 1. Predicting heatLoss @ 5F from hhistoric gas use
from
(1)$$\frac{units}{yr} = \frac{HeatLoss \frac{Btu}{hr} * 24\frac{hr}{day} * Degree \frac{T-day}{yr}}{Design \Delta T * Efficiency * \frac{Btu}{unit}}$$

we get

(9)$$heatLoss \frac{Btu}{hr} = \frac{ \frac{units}{yr} * \frac{Btu}{unit} * \Delta T * eff.  }{24 \frac{hr}{day} * degree \frac{T-day}{yr}}$$

So for 2023, 2022 and 2021 we can get a reasonable guess at heatLoss @5F for 12 Parley Vale.

In [7]:
BtuPerTherm = 100000
gasEff =.9
costPerTherm = 2.33
th12p23 = 595
th12p22 = 680
th12p21 = 608
heatLoss12p23 =  (th12p23 * BtuPerTherm * deltaT * gasEff)/(24 * dd2223)
heatLoss12p22 =  (th12p22 * BtuPerTherm * deltaT * gasEff)/(24 * dd2122)
heatLoss12p21 =  (th12p21 * BtuPerTherm * deltaT * gasEff)/(24 * dd2021)

print(f"{heatLoss12p23 =:,.0f} Btu/hr \
  \n{heatLoss12p22 =:,.0f} btu/hr \
  \n{heatLoss12p21 =:,.0f} btu/hr")

heatLoss12p23 =29,701 Btu/hr   
heatLoss12p22 =33,124 btu/hr   
heatLoss12p21 =29,225 btu/hr



##### 2. Predicting Btu/yr from heatLoss Btu/hr @5F and degreeDays
 
(7)$$\frac{Btu}{yr} = heatLoss \frac{Btu}{hr}*24 \frac{hr}{day} * \frac{1}{ \Delta T}*Degree \frac{T-days}{yr}$$

In [8]:
Btuyr23dd = heatLoss12p23 * 24 / deltaT * dd2223 # 2023
Btuyr22dd = heatLoss12p22 * 24 / deltaT * dd2122 # 2022
Btuyr21dd = heatLoss12p21 * 24 / deltaT * dd2021 # 2021
print(f"2023 {Btuyr23dd = :,.0f} Btu/yr \
      \n2024 {Btuyr22dd = :,.0f} Btu/yr \
      \n2021 {Btuyr21dd = :,.0f} Btu/yr \n")

2023 Btuyr23dd = 53,550,000 Btu/yr       
2024 Btuyr22dd = 61,200,000 Btu/yr       
2021 Btuyr21dd = 54,720,000 Btu/yr 



##### 3. predicted heat pump kwh from HSPF
(5)$$ kwh = \frac{Btu}{(1000*HSPF)}$$

In [9]:

kwh23=Btuyr23dd/(1000 * HSPF)
kwh22=Btuyr22dd/(1000 * HSPF)
kwh21=Btuyr21dd/(1000 * HSPF)
elCost23 =kwh23*costPerKwh
elCost22 =kwh22*costPerKwh
elCost21 =kwh21*costPerKwh
gasCost23 =th12p23 * costPerTherm
gasCost22 =th12p22 * costPerTherm
gasCost21 =th12p21 * costPerTherm

print(f"Assuming {HSPF=} and {costPerKwh = } and {costPerTherm = } \
  \n 2023  {kwh23 = :,.0f} kwh and elCost23 = ${elCost23:,.2f} gasCost23 = ${gasCost23:,.2f}\
  \n 2022  {kwh22 = :,.0f} kwh and elCost22 = ${elCost22:,.2f} gasCost22 = ${gasCost22:,.2f}\
  \n 2021  {kwh21 = :,.0f} kwh and elCost21 = ${elCost21:,.2f} gasCost21 = ${gasCost21:,.2f}")

Assuming HSPF=10.4 and costPerKwh = 0.27 and costPerTherm = 2.33   
 2023  kwh23 = 5,149 kwh and elCost23 = $1,390.24 gasCost23 = $1,386.35  
 2022  kwh22 = 5,885 kwh and elCost22 = $1,588.85 gasCost22 = $1,584.40  
 2021  kwh21 = 5,262 kwh and elCost21 = $1,420.62 gasCost21 = $1,416.64


The PREDICTED energy use using an HSPF 10.4 heat pump system at current energy prices is pretty much a wash; almost exactly the same cost as gas. Hopefully the new system will beat the predictions like apt3 did. Or Maybe the manual J calculations will be lower than the heatLoss@5F derived from historic fuel use. Either way it is a go for new system redesign. 

In [10]:
print(f"Design a system for 12 Parley Vale sized to supply 70F inside temp \
  \nwhen heatLoss@5F ={heatLoss12p22:,.0f} Btu/hr")

Design a system for 12 Parley Vale sized to supply 70F inside temp   
when heatLoss@5F =33,124 Btu/hr


OMG, something is wrong, I currently have a 120,000 Btu boiler! Get over it. think about the boilers at Chestnut Ave. Those old oilfired boilers wer pumpin out...

In [11]:
nozzleSize = .75 # gph
BtuOilBoiler = nozzleSize * BtuPerGal

print(f"Apt 1-4 boilers had {nozzleSize =} gph. @ {BtuPerGal =}. \
It was a {BtuOilBoiler:,.0f}Btu boiler \
    \nfor an apartment that needed {heatLoss:,} Btu on a 5F day")

Apt 1-4 boilers had nozzleSize =0.75 gph. @ BtuPerGal =139000. It was a 104,250Btu boiler     
for an apartment that needed 13,000 Btu on a 5F day


That old boiler was replaced with a 1.5 ton 18,000 Btu Heat Pump

#### COP coefficient of performance

(8)$$COP = \frac{Btu}{kw*3412\frac{Btu}{kw}} =  \frac{powerOut}{powerIn}$$ 

solve for kwh:



While COP is `power_out/power_in` at an instant in time only at a particular outdoor temperatures , we can use to as a rough measure of the relationship of enery_out/ energy_in over a heating season and compare btu_out/kw_in. Given a COP value and knowing the Btu/yr required, we can guess the electrical use for the year(`kw/yr`) and then the `cost/yr`

(9)$$kwh/yr = \frac{Btu/yr}{COP*3412Btu/kw}$$ 

In [12]:
COP = 2
kwhyr = Btuyr22dd/(COP*3412)
costByCOP = kwhyr*costPerKwh
percOfCur = costByCOP/gasCost22
print(f"To create {Btuyr22dd :,.0f} Btu/yr at a COP of {COP:.1f} will require \
    \n   {kwhyr:,.0f} kw/yr at ${costPerKwh:1.2f}/kwh = ${costByCOP:,.2f}/yr \
    \n   which is {percOfCur-1:.0%} more than historic gas costs of ${gasCost22:,.2f}")

To create 61,200,000 Btu/yr at a COP of 2.0 will require     
   8,968 kw/yr at $0.27/kwh = $2,421.45/yr     
   which is 53% more than historic gas costs of $1,584.40


### [SCOP Seasonal Coefficient of PerFormance](https://www.h2xengineering.com/blogs/heat-pump-cop-and-scop-what-they-mean-and-why-they-matter/)

The Seasonal Coefficient of Performance (SCOP) is a metric that measures the energy efficiency of a heat pump over an entire heating season.

Unlike the COP, which provides a snapshot of the heat pump’s efficiency at a specific moment, SCOP takes into account the varying outdoor temperatures and operating conditions throughout the season, giving a more comprehensive picture of the heat pump’s overall performance.

However, below is a simplified example to help you understand the concept of SCOP calculation.

Let’s say we have a heat pump and we want to calculate its SCOP based on the following three temperature ranges during the heating season:

    Temperature range A: -10°C to 0°C
    Temperature range B: 0°C to 10°C
    Temperature range C: 10°C to 20°C
 We also have the heat pump’s COP at the average temperature for each range:

    COP at -5°C (average of range A): 2.0
    COP at 5°C (average of range B): 3.0
    COP at 15°C (average of range C): 4.0
 
Additionally, we need to know the percentage of the heating season the heat pump operates within each temperature range:

    Percentage of time in range A: 20%
    Percentage of time in range B: 50%
    Percentage of time in range C: 30%

To calculate the SCOP, we’ll multiply the COP at each temperature range by the percentage of time the heat pump operates in that range and sum the results:

SCOP = (COP at -5°C x percentage of time in range A) + (COP at 5°C x percentage of time in range B) + (COP at 15°C x percentage of time in range C)

SCOP = (2.0 x 0.2) + (3.0 x 0.5) + (4.0 x 0.3) = 0.4 + 1.5 + 1.2 = 3.1

## products

### Artic

#### Artic 030ZA(BEH2)
    <timwallace@arcticheatpumps.com>
    Tim Wallace
    614.657.8284
    Arctic Heat Pumps
    Michigan and Massachusetts Sales Representative

I was just told the only time a heat exchanger would be necessary is turn of century 1900 registers or water based only registers.
![](./Artic%20030ZA(BEH2).png)

Hi Tim,
 
I looked at the COP ratings of the Artic 030ZA(BEH2) and was underwhelmed. 

For the hydronic house we have talked about I backed into a number for `Btu/yr` from historical therms for gas heating, degree days and efficiency of the gas boiler. I used that with a `COP of 2` to get an idea of what it would cost to run the new system.

While COP is `power_out/power_in` at an instant in time only at a particular outdoor temperatures , we can use to as a rough measure of the relationship of enery_out/ energy_in over a heating season and compare btu_out/kw_in. Given a COP value and knowing the Btu/yr required, we can guess the electrical use for the year(`kw/yr`) and then the `cost/yr`

(9)$$kwh/yr = \frac{Btu/yr}{COP*3412Btu/kw}$$ 

In [13]:
COP = 2
kwhyr = Btuyr22dd/(COP*3412)
costByCOP = kwhyr*costPerKwh
percOfCur = costByCOP/gasCost22
print(f"To create {Btuyr22dd :,.0f} Btu/yr at a COP of {COP:.1f} will require \
    \n   {kwhyr:,.0f} kw/yr at ${costPerKwh:1.2f}/kwh = ${costByCOP:,.2f}/yr \
    \n   which is {percOfCur-1:.0%} more than historic gas costs of ${gasCost22:,.2f}")


To create 61,200,000 Btu/yr at a COP of 2.0 will require     
   8,968 kw/yr at $0.27/kwh = $2,421.45/yr     
   which is 53% more than historic gas costs of $1,584.40



[arctic specs suck](https://www.arcticheatpumps.com/specifications.html )
letters to Arctic

My use for your '''air to water heat pump products''' will be limited to '''DHW''' and the '''existing kitchen/bath radiant loop'''. That might extend to some kind of wall '''radiant emitters''' for the other two bathrooms depending on calculating the load and the availability of products matching the load. My needs most closely match the following configuration.

<img src="https://cdn-fdbpa.nitrocdn.com/gKOxVeMEjzbIXyXQDzDOXkUdVDxCjtsJ/assets/images/optimized/rev-a08ae6a/www.arcticheatpumps.com/images/2022/01/26/h-w1.jpg" alt="Girl in a jacket" width="800">

I will set up some monitoring on the existing radiant loop for temperature-in and temperature-out which combined with knowing the flow rate should give me the btu/hr load.

I am not looking for a "quick estimate". What I need from you is performance and engineering data and pricing on all of listed components. 

The rest of the house will likely replace 180 degree baseboard emitters with air to air heat pumps connected to some combination of wall units and air handlers. 

29 years ago this was a carefully designed system  which heats a 2,700 sf house with high ceilings and lots of windows for ~$1,500/yr @ $2.33/therm. I even got John Siegenthaler to weigh in with some invaluable advice. 

With electricity here @ $0.27/kwh and natural gas @ .07/kwh it will be a big challenge to design a replacement system. Whatever it is will have to spend most of its time in the operating range that gives you a COP of ~4 or else the operating cost penalty will be too great. 

I am licensed builder in MA with EPA 608 universal lic. and am a member of the MassSave Heat Pump Installer Network. So far I have had very good results replacing oil steam boilers with cold climate heat pumps connected to air handlers delivering heat through exposed spiral ductwork running in the conditioned space. Tenants are spending almost half of what oil cost to run their systems.

### LG

#### [ LG launches Therma V R290 Monobloc propane AWHP](https://www.pv-magazine.com/2023/11/29/lg-launches-residential-air-to-water-propane-heat-pump/#:~:text=LG%20is%20offering%20the%20heat,use%20of%20residential%20PV%20systems.)

[LG blog - R290 Heat Pumps: A Sustainable Solution for Heating and Cooling](https://www.lg.com/global/business/hvac-blog/introducing-therma-v-r290-monobloc)
* a new version of The Therma V R290 Monobloc heat pump
* flow temp 75C (167F)
* 100% output @ -15C (5F)
* uses propane (R290)
* LG is offering the heat pump in four versions with nominal capacities of 9 kW, 12 kW, 14 kW and 16 kW.
* Its seasonal coefficient of performance (SCOP) is reportedly over 5.

Monobloc is all in one solution without additional indoor unit. It’s only connected to water tank so easy and simple to be installed. Contact us for this sustainable heat pump system. At low ambient temperatures down to -15℃ (5F), the Monobloc delivers powerful heating at 100% capacity

# from wiki2
## oversizing is not OK
from [Energy Vanguard - Allison Bailes](https://www.energyvanguard.com/blog/can-you-oversize-a-mini-split-heat-pump/ )

#### Understanding turndown
For example, the Mitsubishi FS06 wall-mounted ductless unit has a rated cooling capacity of 6,000 BTU per hour and goes down to a minimum capacity of 1,700 BTU per hour.  That’s a turndown percentage of 28% (1,700 ÷ 6,000).  If you put that unit in a room with a load of 1,500 BTU per hour, it will bottom out nearly 100% of the time it’s running in cooling mode.

Sometimes you’ll see turndown ratio, which is the same concept as turndown percentage but upside down.  It’s the ratio of maximum to minimum capacity.  For the example here, it would be 6,000 ÷ 1,700 = 3.5.  So the turndown ratio is 3.5 to 1, sometimes written 3.5:1.

#### What happens at part-load?
One of the great advantages of using mini-splits is their variable capacity.  By oversizing them, you can lose that advantage completely.  That 6,000 BTU per hour mini-split in a room with 1,500 BTU per hour of cooling load doesn’t act at all like it has variable capacity because it’s always running at the lowest capacity except on the really extreme temperature days.

When the bottom end of the capacity range is at or above the heating or cooling load, there’s no benefit in part-load conditions.  As the heating or cooling load drops from the design load, you want a system whose capacity also drops.

That 6,000 BTU/hr system with a bottom end of 1,700 BTU/hr serving a load of 1,500 BTU/hr doesn’t vary at all in part-load conditions.

When the load is 1,500 BTU/hr (the design load), the system will give you 1,700 BTU/hr.  When the load drops to 1,000 BTU/hr, the system will give you 1,700 BTU/hr.  When the load is only 500 BTU/hr, the system will give you 1,700 BTU/hr.

Yes, oversizing is a problem

So, yes, you absolutely can oversize a mini-split.  And when you do, you end up with some of the same problems you get from oversizing conventional systems:  poor humidity control, short cycling, and wasted money.

====HSPF====
HSPF is defined as the ratio of heat output (measured in BTUs) over the heating season to electricity used (measured in watt-hours).[1][2] It therefore has units of BTU/watt-hr.

The higher the HSPF rating of a unit, the more energy efficient it is. An electrical resistance heater, which is not considered efficient, has an HSPF of 3.41.[3]

Depending on the system, an HSPF ≥ 9 can be considered high efficiency and worthy of a US Energy Tax Credit.[4]

calculated over a year
 HSPF = BTU/watt-hr
 HSPF = BTU/(1000*kwh)

For instance, a system which delivers an HSPF of 7.7 will transfer 2.25 times as much heat as electricity consumed over a season.[5]

## AWHP


https://www.pmengineer.com/articles/92614-cold-climate-air-to-water-heat-pumpspart-2

https://www.pmengineer.com/ext/resources/PME/2016/July/pme0716Siggy_Figure4.png


email to Artic on air source hydronic

I am interested in Arctic Heat Pump 035ZA/BE – 29,000 BTU for use in a domestic HW + radiant application. Could you send me submittal sheets and any other details on 

[ Arctic Air to Water Heat Pumps](https://1drv.ms/b/s!Apj4JFLjNWyohbVblBSgPTh_KsYWgA?e=H0SkNv)

#### Domestic Hot Water
The AltSource tank is equipped with an optional internal Heat Exchanger (HX) for domestic hot water 
production. This consists of 200 feet of 1/2” copper piping. Based on 120°F water tank temperatures this 200 ft of ½” OD cooper pipe, 26.2 sq/ft surface area and has a volume of 1.5 total gallons.

Because the efficiency of the heat pump is better at lower water temperatures, we recommend using the internal domestic hot water coil in a pre-heat arrangement. In this diagram the cold water enters the coil from the bottom and exists at about 95% of the tank setting temperature. Following this the warm water then goes to the primary tank or an on-demand water tank where it is topped up a few degrees to reach 120°F or more depending on the application. When sizing a heat pump with domestic hot water load please add about 10,000 BTU per person to the daily load. So, a house with 4 persons would need an extra 40,000 BTU per day. On an hourly bases this would be 1,667 BTU/hour over 24 hours.

What is a flow rate and what should I ask for? Consider how many of these needs you may have simultaneously to determine the maximum flow rate you require. For a home with 1 bathroom, we recommend 6-7 GPM; 2 bathrooms, 8 – 9 GPM; and 3+ bathrooms, 9-11 GPM.
What is a flow rate and what should I ask for? Consider how many of these needs you may have simultaneously to determine the maximum flow rate you require. For a home with 1 bathroom, we recommend 6-7 GPM; 2 bathrooms, 8 – 9 GPM; and 3+ bathrooms, 9-11 GPM.

https://plumbestore.com/products/eco-king-52-stainless-steel-coil-tank?variant=40820050100277&currency=USD&utm_medium=product_sync&utm_source=google&utm_content=sag_organic&utm_campaign=sag_organic&srsltid=Ad5pg_GvtvxwqGw-r-NdmLPi-dtNOxGVEkE9--PY0lQMW7D3lHV6h7GzU2s

####  heat pump water heater for HW and radiant ====
http://www.civil.uwaterloo.ca/beg/archtech/hot_water_heater_furnace.pdf

like this but with heat pump water heater instead
[[File:Heat pump for DHW+radiant.png]]

#### hybrid heat pump algorithms for resis.
https://www.greenbuildingadvisor.com/question/rheem-hybrid-pre-warm-mode-while-in-heat-pump-mode

https://www.nrel.gov/docs/fy14osti/52635.pdf
